# 함수 데커레이터와 클로저

## 7.1 데커레이터 기본 지식

```python
@decorate
def target():
    print('running target()')
```
위 코드는 아래의 코드와 동일하게 작동한다.    

```python
def target():
    print('running target()')
   
target = decorate(target)
```

In [1]:
# 예제 7-1 일반적으로 데커레이터는 함수를 다른 함수로 대체한다.
def deco(func):
    def inner():
        print('running inner()')
    return inner # inner 함수 객체를 반환

@deco
def target(): # target을 deco로 데커레이트 함
    print('running target()')

target() # 데커레이트된 target()을 호출하면 실제로는 inner()를 실행한다.
target # target이 inner()를 가리키고 있음

running inner()


<function __main__.deco.<locals>.inner()>

## 7.2 파이썬이 데커레이터를 실행하는 시점

데커레이터는 파이썬이 모듈을 로딩하는 시점인 임포트 타임에 실행된다.    

```python
# registration.py

registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3(): # 데커레이트 되지 않은 f3
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()
```

In [2]:
!python registration.py

running register(<function f1 at 0x7f3e066a8b00>)
running register(<function f2 at 0x7f3e066a8a70>)
running main()
registry -> [<function f1 at 0x7f3e066a8b00>, <function f2 at 0x7f3e066a8a70>]
running f1()
running f2()
running f3()


register()는 모듈 내의 다른 어떠한 함수보다 먼저 실행된다.   
스크립트로 실행하지 않고 임포트 할 때는 아래와 같이 출력된다.

In [3]:
import registration

running register(<function f1 at 0x7f7804698b00>)
running register(<function f2 at 0x7f7804698cb0>)


registry에는 아래와 같은 내용이 들어있다.

In [4]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

이를 통해 데커레이터는 모듈이 임포트되자마자 실행되지만(임포트 타임), 데커레이트된 함수는 명시적으로 호출될 때만 실행되는 것을 알 수 있다.(런타임)

## 7.3 데커레이터로 개선한 전략 패턴

등록 데커레이터를 이용해 6.1절의 프로모션 할인 코드를 개선할 수 있다.    
가장 큰 문제인 가장 큰 할인 방식을 찾는 best_promo() 함수에 사용되는 promos 리스트에 새로운 함수명을 추가를 하지 않는 문제를 막을 수 있다.

In [5]:
# 예제 7-3 promotion 데커레이터로 채운 promos 리스트
promos = []

# promotion() 데커레이터는 promo_func promos에 추가하고 그대로 반환한다.
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_itme(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

# @promotion덕분에 best_promo는 변경 할 필요가 없다
def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

## 7.4 변수 범위 규칙

In [6]:
# 예제 7-4 지역 및 전역 변수를 읽는 함수
def f1(a):
    print(a)
    print(b)

try:
    f1(3)
except NameError:
    print("NameError: name 'b' is not defined")

3
NameError: name 'b' is not defined


위의 경우에는 에러가 발생하지만, 아래와 같이 사용하면 제대로 작동한다.

In [7]:
b = 6
f1(3)

3
6


함수 안에 있는 b 변수가 출력 전에 선언되었기 때문에 에러가 발생한다.

In [8]:
# 예제 7-5 함수 본체 안에서 값을 할당하기 때문에 지역 변수가 되는 b
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

try:
    f2(3)
except UnboundLocalError:
    print("UnboundLocalError: local variable 'b' referenced before assignment")

3
UnboundLocalError: local variable 'b' referenced before assignment


b를 전역 변수로 다루기를 원하면 global를 이용한다.

In [9]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)
print(b)
f3(3)
b = 30
b

3
6
9
3
9


30

#### _ 바이트코드 비교 _

In [10]:
# 예제 7-6 [예제 7-4] f1() 함수의 디스어셈블리
from dis import dis
dis(f1)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


전역명 b를 로딩하는 것을 알 수 있다.

In [11]:
# 예제 7-7 [예제 7-5] f2() 함수의 디스어셈블리
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


지역명 b를 로딩하는 것을 알 수 있다.

## 7.5 클로저

이전의 값을 기억하고 있는 avg()함수를 생성해보자. 먼저 클래스를 이용해 구현한다.

In [12]:
# 예제 7-8 average_oo.py: 이동 평균을 계산하는 클래스
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [13]:
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


고위함수 make_averager()를 이용해 구현한다.

In [14]:
# 예제 7-9 average.py: 이동 평균을 계산하는 고위 함수
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

make_averager()가 호출도면 average() 함수 객체를 반환하고, average() 함수는 호출될 때 마다 받은 인수를 series에 추가한다.

In [15]:
# 예제 7-10 [예제 7-9] 테스트
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [16]:
# 예제 7-11 [예제 7-9]의 make_averager()로 생성한 함수 조사하기
print(avg.__code__.co_varnames)
print(avg.__code__.co_freevars)

('new_value', 'total')
('series',)


In [17]:
# [예제 7-10]에서 계속
print(avg.__code__.co_freevars)
print(avg.__closure__)
print(avg.__closure__[0].cell_contents)

('series',)
(<cell at 0x7f780463fc50: list object at 0x7f7804679af0>,)
[10, 11, 12]


## 7.6 nonlocal 선언